In [ ]:
using ITensors: MPO, MPS, OpSum, expect, inner, siteinds 
using ITensors
using ITensorMPS
using Plots
using QuanticsTCI
import TensorCrossInterpolation as TCI
using Quantics
using HDF5 
include("MPO_SCF_normal.jl") 
include("get_functions.jl")


In [ ]:
U_0 = 4
U_1 = 0.5

#just super moire
function novel_potential(L)
    k_1 = 2 * pi / (5* sqrt(2))
    k_2 = 8 * pi / (  2^9* sqrt(3))
    xvals = range(0, (2^L - 1); length=2^L)
    
    f(x) =  U_1 * cos(k_1 * (x - 2^9)) + U_0 +  U_1 * cos(k_2 * (x - 2^9)) 
 
    
    
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return density_mpo 
end

"""
#for domain wall
function novel_potential(L)
    k_1 = 2 * pi / (5* sqrt(2))
    k_2 = 8 * pi / (  2^29* sqrt(3))
    delta = 0.2
    W = (2^30)/40
    xvals = range(1, (2^L  ); length=2^L)
    
    f(x) =  U_1 * cos(k_1 * (x - 2^29)) + U_0 +  U_1 * cos(k_2 * (1 + delta * tanh((x- 2^29)/W)) * (x- 2^29)) 
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return density_mpo 
end
"""

#Antiferro guess
function novel_initial_guess_up(L)
    
    xvals = range(0, (2^L - 1); length=2^L)
    #anti-ferro, half-filling
    f(x) = (x )%2
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt,density_mpo,density_mps 
end

function novel_initial_guess_down(L)
    
    xvals = range(0, (2^L - 1); length=2^L)
    #anti-ferro, half-filling
    f(x) = (x+1)%2
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt,density_mpo,density_mps
end

In [ ]:
L = 30
N = 200

#making all calculations with same siteinds, flexible to modify
f = h5open("template_mps.h5","r")
templ = read(f,"template_mps",MPS)
close(f)
ini_rand = templ;
sites = siteinds(ini_rand);

id = OpSum()
for j = 1:L
    id += 1,"Id",j
end

#not divided by L here, normalized in the SCF.jl file
Id_op = MPO(id,sites); 

u_mpo = novel_potential(L);
qtt_den_old_up, initial_den_up,ini_mps_up = novel_initial_guess_up(L);
qtt_den_old_down, initial_den_down,ini_mps_down  = novel_initial_guess_down(L);

In [ ]:
max_iter = 1000
@time qtt_den_new_up,qtt_den_new_down, Tn_list_up,Tn_list_down, conv_errr  = SCF_Hubbard(L,sites,u_mpo,max_iter,N,
1e-3,initial_den_up,initial_den_down,ini_mps_up,ini_mps_down,qtt_den_old_up
,qtt_den_old_down, 1);
get_error_plot(conv_errr)

In [ ]:
#save Tn for later usage 
folder_name = "tn_billion_200cheby_domain_diag_up"
 
if !isdir(folder_name)
    mkdir(folder_name)
    println("folder '$folder_name' is created")
else
    println("folder '$folder_name' exists")
end

for (i, hampo) in enumerate( Tn_list_up)

    file_path = joinpath(folder_name, "myfilempo_$i.h5")
    
    h5open(file_path, "w") do f
        write(f, "hampo",hampo)
    end
 
end

In [ ]:
#save Tn for later usage 
folder_name = "tn_billion_200cheby_domain_diag_down"
 
if !isdir(folder_name)
    mkdir(folder_name)
    println("folder '$folder_name' is created")
else
    println("folder '$folder_name' exists")
end

for (i, hampo) in enumerate( Tn_list_up)

    file_path = joinpath(folder_name, "myfilempo_$i.h5")
    
    h5open(file_path, "w") do f
        write(f, "hampo",hampo)
    end
 
end